A little bit of segue on the previous notebook, I've recreated another jupyter to explore more on matrix factorization for collaborative filtering without the use of TF (did gone overboard with previous notebook)

Goal in mind being using numpy and scipy for mf svd or nmf with a little bit more control on matrix factorization for different federated clients as I'll explore later.

In [1]:
from src.data_examples.ex2_data_loader import ExampleDataLoader

from enum import Enum, auto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import svds as scipy_svds

pd.options.mode.chained_assignment = None

In [2]:
RAND_SEED = 1337

In [3]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean().merge()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [4]:
# to trucate any other features outside of matrix factorization
# select only user_id, movie_id, and rating for our matrix

features_X = ['user_id', 'movie_id']
feature_Y = 'rating'

df_main = data_loader.df[features_X + [feature_Y]]


for feature in features_X:
  df_main[feature] = df_main[feature].astype('int32')

df_main

,user_id,movie_id,rating
0,195,241,3.0
1,304,241,5.0
2,5,241,4.0
3,233,241,4.0
4,62,241,3.0
...,...,...,...
99995,862,1678,3.0
99996,862,1677,1.0
99997,862,1679,2.0
99998,895,1680,3.0


In [5]:
def df_to_np_sparse_matrix(df, U_c, V_c, val_c):
  # pivot the data with movie_id as column to create sparse mxn matrix 
  df = df[[U_c, V_c, val_c]]
  df_pivot = df.pivot(index=U_c, columns=V_c, values=val_c).fillna(0)
  return df_pivot.values

def sparse_matrix_normalize(mat_sparse):
  # normalize matrix from means of axis 1
  mat_mean = np.mean(mat_sparse, axis=1)
  return (mat_sparse - mat_mean.reshape(-1, 1)), mat_mean

def eval_svd_factorization(df, U_c, V_c, val_c, latent_factor=25):
  mat_sparse = df_to_np_sparse_matrix(df_main, 'user_id', 'movie_id', 'rating')
  mat_sparse_norm, mat_sparse_mean = sparse_matrix_normalize(mat_sparse)

  U, sigma, Vt = scipy_svds(mat_sparse, k=latent_factor)
  sigma = np.diag(sigma)

  mat_prediction = np.dot(np.dot(U, sigma), Vt) + mat_sparse_mean.reshape(-1, 1)

  return mat_prediction, (U, sigma, Vt)


In [6]:
mat_pred, (U, sigma, Vt) = eval_svd_factorization(df_main, 'user_id', 'movie_id', 'rating', latent_factor=25)

df_pred = pd.DataFrame(mat_pred) 
df_pred # which will directly encodes user_id as index axis 1 and movie_id as columns

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5.303524,2.945366,1.985449,3.256721,1.494244,1.391887,5.688369,2.901966,4.483418,2.491556,...,0.567039,0.600525,0.575460,0.578250,0.611382,0.583603,0.583151,0.583377,0.604631,0.674156
1,2.224920,0.131546,0.078819,0.504417,0.167155,0.469432,1.876868,0.664002,2.676077,0.764084,...,0.139687,0.115852,0.137310,0.137121,0.136788,0.141843,0.152044,0.146943,0.132940,0.103478
2,0.482169,0.124535,0.148999,-0.109686,-0.017135,0.076274,-0.236408,0.063128,-0.412632,0.181921,...,0.091629,0.074502,0.121112,0.110666,0.080249,0.102420,0.127711,0.115065,0.103400,0.093294
3,0.407867,-0.134845,0.165022,0.008629,0.064627,0.096473,0.415217,-0.032686,-0.184857,-0.113599,...,0.067801,0.048875,0.057164,0.058720,0.063661,0.067104,0.077650,0.072377,0.056577,0.059538
4,4.293486,1.567618,0.674660,1.841681,0.764771,0.112497,3.155563,1.677952,-0.289173,0.755384,...,0.290336,0.297954,0.277645,0.284780,0.271738,0.300080,0.302144,0.301112,0.308988,0.288928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,2.790540,0.203940,0.202682,0.094615,0.583741,0.184341,1.534100,0.604107,1.771846,0.236496,...,0.131297,0.131142,0.147277,0.139604,0.136981,0.119975,0.111411,0.115693,0.124755,0.114675
939,1.296988,0.267445,0.149078,2.488111,0.695570,0.251902,3.079583,2.173673,1.874501,0.048981,...,0.229078,0.260641,0.197695,0.205122,0.222456,0.223537,0.230659,0.227098,0.189555,0.182243
940,2.219198,0.078794,0.396681,0.041805,0.028757,0.044413,1.830630,0.513444,0.960988,0.180768,...,0.041240,0.043262,0.036469,0.041950,0.058309,0.050131,0.044566,0.047348,0.063723,0.052194
941,1.187992,0.265697,-0.067663,0.125764,0.059492,0.148631,0.215381,1.810062,0.287538,0.054384,...,0.228651,0.231076,0.208217,0.205597,0.198821,0.207480,0.221727,0.214603,0.206111,0.177924


In [7]:
tgt_user_id = 420
n_movies = 15

pred_movie_ids = df_pred.iloc[tgt_user_id].sort_values(ascending=False)[:n_movies].index

# a movie string formatter, note: uses dataloader.df_movies
fmt_movie_printout = lambda movie_id: f'movie_id: {movie_id}'.ljust(17) + data_loader.df_movies.loc[movie_id]['title']

pred_movie_ids = df_main[df_main['user_id'] == tgt_user_id]['movie_id'] [:n_movies]
print('recommendtion')
for movie_id in pred_movie_ids:
  print(fmt_movie_printout(movie_id))

pred_movie_ids


recommendtion
movie_id: 237    Raising Arizona (1987)
movie_id: 172    Princess Bride, The (1987)
movie_id: 268    Full Monty, The (1997)
movie_id: 301    L.A. Confidential (1997)
movie_id: 332    Game, The (1997)
movie_id: 116    Rock, The (1996)
movie_id: 99     Fargo (1996)
movie_id: 78     Fugitive, The (1993)
movie_id: 97     Silence of the Lambs, The (1991)
movie_id: 55     Pulp Fiction (1994)
movie_id: 11     Usual Suspects, The (1995)
movie_id: 330    Edge, The (1997)
movie_id: 95     Terminator 2: Judgment Day (1991)
movie_id: 175    Aliens (1986)
movie_id: 207    Young Frankenstein (1974)


1007     237
5660     172
6192     268
6523     301
8476     332
10247    116
12017     99
14192     78
15887     97
16276     55
18593     11
19589    330
21931     95
22948    175
24126    207
Name: movie_id, dtype: int32

## Federated Learning with Clients (or Silos) Manager

In [8]:
global_latent_factor = 50

In [9]:
class DBKind(Enum):
  shared = auto()
  private = auto()

In [10]:
# crude emulation server-clients with ClientManager

class Client:
  def __init__(self, _uuid=None):
    self.__task_pool = []
    self.__private_dbs = {}
    self.__shared_dbs = {}
    self.__named_variables = {}
    if _uuid is None:
      self._uuid = id(self) # fallbacks to basically the address on memory of client object for the sake of simplicity
    else:
      self._uuid = _uuid

  # client uuid
  def get_uuid(self):
    return self._uuid

  # client tasks
  def clear_tasks(self):
    self.__task_pool.clear()

  def add_task(self, fn):
    self.__task_pool.append(fn)

  def run_tasks(self):
    for task in self.__task_pool:
      task(self)

  # client public named variables 
  def clear_named_variables(self):
    self.__named_variables.clear()

  def set_named_variable(self, name, value):
    self.__named_variables[name] = value

  def get_named_variable_value(self, name):
    return self.__named_variables.get(name)

  def get_named_variable_value_names(self):
    return self.__named_variables.keys()

  # client database connection (read: dataframe)
  def add_db(self, kind: DBKind, name, db):
    if kind == DBKind.shared:
      self.__shared_dbs[name] = db
    elif kind == DBKind.private:
      self.__private_dbs[name] = db

  def get_db(self, kind: DBKind, name):
    if kind == DBKind.shared:
      return self.__shared_dbs[name]
    elif kind == DBKind.private:
      return self.__private_dbs[name]


class ClientManager:
  def __init__(self):
    self._client_bucket = []
    self._client_uuids = []

  def new_client(self, _uuid=None):
    client = Client(_uuid=_uuid)
    self._client_bucket.append(client)
    self._client_uuids.append(client.get_uuid())
    if _uuid is None:
      client._uuid = id(client)
    else:
      client._uuid = _uuid
    return client

  def get_client_uuids(self):
    return self._client_uuids

  def get_client_by_uuid(self, _uuid):
    if _uuid not in self._client_uuids:
      return None
    return self._client_bucket[self._client_uuids.index(_uuid)]

  def force_run_tasks(self):
    for client in self._client_bucket:
      client.run_tasks()

  def force_clear_tasks(self):
    for client in self._client_bucket:
      client.clear_tasks()
    
  def force_clear_named_variabless(self):
    for client in self._client_bucket:
      client.clear_named_variables()

  def run_callback_on_clients(self, fn: callable, *args, **kwargs):
    for client in self._client_bucket:
      fn(client, *args, **kwargs)


# example
manager = ClientManager()
for _ in range(4):
  client = manager.new_client()
  client.add_task(lambda this: print(f'running task on client: printing uuid {this.get_uuid()}'))

print('client uuids:', manager.get_client_uuids())
manager.force_run_tasks()

client uuids: [2544869531728, 2544869532640, 2544869533168, 2544869533264]
running task on client: printing uuid 2544869531728
running task on client: printing uuid 2544869532640
running task on client: printing uuid 2544869533168
running task on client: printing uuid 2544869533264


### try federated learining from clusters of zipcodes

In [11]:
unique_zip_codes = data_loader.df_users['zip_code'].unique()

# change to len(unique_zip_codes) for individual zip codes to be assigned as individual client
n_zip_code_clusters = 20

unique_zip_codes_clusters = np.array_split(unique_zip_codes, n_zip_code_clusters)

In [12]:
unique_zip_codes = data_loader.df_users['zip_code'].unique()
user_ratings_headers = ['zip_code', 'user_id', 'movie_id', 'rating']

manager = ClientManager()

for zip_codes in unique_zip_codes_clusters:
  uuid_name = '-'.join(zip_codes[:3]) + '...'
  client_ref = manager.new_client(_uuid=uuid_name)
  client_ref.add_db(kind=DBKind.shared, name='df_movies', db=data_loader.df_movies)

  zip_code_mask = data_loader.df['zip_code'].apply(lambda x: x in zip_codes)
  private_df = data_loader.df[zip_code_mask]
  private_df = private_df[user_ratings_headers].copy()
  client_ref.add_db(kind=DBKind.private, name='df_user_ratings', db=private_df)


In [13]:
# inspec one of the client private user->ratings dataset

__client_ref = manager.get_client_by_uuid(manager.get_client_uuids()[0])
__client_ref.get_db(DBKind.private, 'df_user_ratings')

,zip_code,user_id,movie_id,rating
0,55105,195,241,3.0
2,98101,5,241,4.0
8,29206,12,241,2.0
13,37212,17,241,5.0
14,85711,0,241,5.0
...,...,...,...,...
99702,10003,384,1535,5.0
99703,10003,384,1534,4.0
99891,78741,726,1614,1.0
99907,55105,652,1619,2.0


### adding svd fit tasks to each clients

In [14]:
# force clear task to prevent flooding of print statement or dupicate tasks in pool
manager.force_clear_tasks()

In [15]:
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  client_ref.add_task(
    lambda client: print(f'running task on client "{client.get_uuid()}" with num of participating users: {client.get_db(DBKind.private, "df_user_ratings")["user_id"].unique().shape[0]}')
  )

  client_ref.set_named_variable('latent_factor', global_latent_factor)

# try to run newly created print tasks on client
manager.force_run_tasks()


running task on client "85711-94043-32067..." with num of participating users: 66
running task on client "17870-20854-46260..." with num of participating users: 63
running task on client "20003-46005-89503..." with num of participating users: 46
running task on client "20009-07039-60115..." with num of participating users: 58
running task on client "53705-30067-78750..." with num of participating users: 52
running task on client "61401-55345-11231..." with num of participating users: 45
running task on client "39042-77005-77801..." with num of participating users: 44
running task on client "63108-55439-77904..." with num of participating users: 48
running task on client "76059-45660-61455..." with num of participating users: 46
running task on client "83686-96819-44092..." with num of participating users: 46
running task on client "85282-33308-53202..." with num of participating users: 47
running task on client "92121-21012-V5A2B..." with num of participating users: 48
running task on 

In [16]:
def task_eval_svd_factorization(client):
  print('begin factorization ...', end=' ')

  df = client.get_db(DBKind.private, 'df_user_ratings')
  latent_factor = client.get_named_variable_value('latent_factor')

  mat_pred, (U, sigma, Vt) = eval_svd_factorization(df, 'user_id', 'movie_id', 'rating', latent_factor=latent_factor)

  client.set_named_variable('local_prediction_matrix', mat_pred)
  client.set_named_variable('U', U)
  client.set_named_variable('sigma', sigma)
  client.set_named_variable('Vt', Vt)

  print('finished factorization\n')


for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)
  client_ref.add_task(task_eval_svd_factorization)

In [17]:
# run all factorization tasks
manager.force_run_tasks()

running task on client "85711-94043-32067..." with num of participating users: 66
begin factorization ... finished factorization

running task on client "17870-20854-46260..." with num of participating users: 63
begin factorization ... finished factorization

running task on client "20003-46005-89503..." with num of participating users: 46
begin factorization ... finished factorization

running task on client "20009-07039-60115..." with num of participating users: 58
begin factorization ... finished factorization

running task on client "53705-30067-78750..." with num of participating users: 52
begin factorization ... finished factorization

running task on client "61401-55345-11231..." with num of participating users: 45
begin factorization ... finished factorization

running task on client "39042-77005-77801..." with num of participating users: 44
begin factorization ... finished factorization

running task on client "63108-55439-77904..." with num of participating users: 48
begin fa

In [18]:
# fetch all client saved variables as weakrefs, stored in 'server' as dict() notebook local variable in this case

clients_named_variables = {}
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  clients_named_variables[client_uuid] = {}

  for key in ['sigma']: # 'U', 'Vt'
    val = client_ref.get_named_variable_value(key)
    clients_named_variables[client_uuid][key] = val

In [19]:
# compiling sigma diag matrixes with averatge aggregation based on number of clients

sigmas = [i['sigma'] for i in clients_named_variables.values()]

sigma_avg = sigmas.pop(0)
for sig in sigmas:
  sigma_avg += sig

sigma_avg /= len(sigmas)


# ship or broadcast aggregated sigma values to clients

for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)
  client_ref.set_named_variable('sigma_agg', sigma_avg)


# inspec aggregated sigma values
sigma_avg, sigma_avg.shape

(array([[ 62.21772382,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,  62.53342105,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,  62.6465368 , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ..., 229.3118155 ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
         257.72246912,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        , 674.35118165]]),
 (50, 50))

### perform inference with first instance of user on each cluster/client  

In [20]:
def callback_predict(client):
  n_movies = 5
  user = client.get_db(DBKind.private, 'df_user_ratings')['user_id'].iloc[0] # for now uses first instance of the user on each client
  
  print('getting recommendation for user:', f'{user}', f', in cluster "{client.get_uuid()}"')

  U = client.get_named_variable_value('U')
  sigma = client.get_named_variable_value('sigma_agg')
  Vt = client.get_named_variable_value('Vt')

  new_local_mat_pred = np.dot(np.dot(U, sigma), Vt) 
  # i'll skip the mean addition at 'the mean time' since addition is linear operation and should not affect sorting of the result (hopefully)

  df_pred = pd.DataFrame(new_local_mat_pred)
  
  pred_movie_ids = df_pred.iloc[int(user)].sort_values(ascending=False)[:n_movies].index

  for n, movie_id in enumerate(pred_movie_ids):
    print(f'No.{n+1}'.ljust(5), fmt_movie_printout(movie_id))
  print()

manager.run_callback_on_clients(callback_predict)

getting recommendation for user: 195 , in cluster "85711-94043-32067..."
No.1  movie_id: 285    English Patient, The (1996)
No.2  movie_id: 268    Full Monty, The (1997)
No.3  movie_id: 237    Raising Arizona (1987)
No.4  movie_id: 284    Secrets & Lies (1996)
No.5  movie_id: 152    Fish Called Wanda, A (1988)

getting recommendation for user: 62 , in cluster "17870-20854-46260..."
No.1  movie_id: 285    English Patient, The (1996)
No.2  movie_id: 99     Fargo (1996)
No.3  movie_id: 257    Contact (1997)
No.4  movie_id: 49     Star Wars (1977)
No.5  movie_id: 268    Full Monty, The (1997)

getting recommendation for user: 89 , in cluster "20003-46005-89503..."
No.1  movie_id: 317    Schindler's List (1993)
No.2  movie_id: 212    Room with a View, A (1986)
No.3  movie_id: 196    Graduate, The (1967)
No.4  movie_id: 285    English Patient, The (1996)
No.5  movie_id: 99     Fargo (1996)

getting recommendation for user: 144 , in cluster "20009-07039-60115..."
No.1  movie_id: 97     Silenc